# Prepare experimentation

In [7]:
import pandas as pd
import io

EXPERIMENT_ID = "exp004"
TRAIN_DATA_FILE = "artifacts/data/all_train.csv"
BACKTESTING_STRATEGY = "train-val"
TEST_SIZE_RATIO = 0.2
BACKTESTING_VAL_RATIO = 0.1
BACKTESTING_PERF_RATIO = 0.1
FINAL_VAL_RATIO = 0.1
TARGET_COLUMN = "Sentiment"

## Generate splits

In [ ]:
!uv run cli generate-splits \
    --experiment-id $EXPERIMENT_ID \
    --input-file $TRAIN_DATA_FILE \
    --backtesting-strategy $BACKTESTING_STRATEGY \
    --test-size $TEST_SIZE_RATIO \
    --backtesting-val-size $BACKTESTING_VAL_RATIO \
    --perf-estimation-val-size $BACKTESTING_PERF_RATIO \
    --final-model-val-size $FINAL_VAL_RATIO

In [2]:
!ls -lah artifacts/experiments/exp004
!ls -lah artifacts/experiments/exp004/backtesting
!ls -lah artifacts/experiments/exp004/performance_estimation/
!ls -lah artifacts/experiments/exp004/final_model

total 296
drwxr-xr-x@ 6 witjakuczunpriv  staff   192B Oct 18 12:22 .
drwxr-xr-x@ 7 witjakuczunpriv  staff   224B Oct 18 12:38 ..
drwxr-xr-x@ 4 witjakuczunpriv  staff   128B Oct 18 12:42 backtesting
drwxr-xr-x@ 4 witjakuczunpriv  staff   128B Oct 18 12:22 final_model
drwxr-xr-x@ 4 witjakuczunpriv  staff   128B Oct 18 12:22 performance_estimation
-rw-r--r--@ 1 witjakuczunpriv  staff   146K Oct 18 12:22 test.csv
total 8
drwxr-xr-x@ 4 witjakuczunpriv  staff   128B Oct 18 12:42 .
drwxr-xr-x@ 6 witjakuczunpriv  staff   192B Oct 18 12:22 ..
-rw-r--r--@ 1 witjakuczunpriv  staff   648B Oct 18 13:00 results.json
drwxr-xr-x@ 4 witjakuczunpriv  staff   128B Oct 18 12:22 train-val
total 1176
drwxr-xr-x@ 4 witjakuczunpriv  staff   128B Oct 18 12:22 .
drwxr-xr-x@ 6 witjakuczunpriv  staff   192B Oct 18 12:22 ..
-rw-r--r--@ 1 witjakuczunpriv  staff   521K Oct 18 12:22 train.csv
-rw-r--r--@ 1 witjakuczunpriv  staff    60K Oct 18 12:22 val.csv
total 1464
drwxr-xr-x@ 4 witjakuczunpriv  staff   128B Oct 18

# Develop models...

In [ ]:
!uv run cli list-models

2025-10-18 16:53:50.613 | INFO     | mlops.app:list_models:134 - Available models:
- bielik-1.5B
- svm-opt
- roberta-base
- bert-base
- svm-base
- bert-micro-base
- roberta-base-long
- bert-micro-long
- gemma-3-1b-it


# Run experiments

## SVM base

In [4]:
SVM_BASE_CFG = "svm-base"

In [ ]:
!uv run cli run-backtesting \
    --experiment-id $EXPERIMENT_ID \
    --model-config-name $SVM_BASE_CFG

## SVM with hiperparameter optimization

In [ ]:
SVM_OPT_CFG = "svm-opt"

In [ ]:
!uv run cli run-backtesting \
    --experiment-id $EXPERIMENT_ID \
    --model-config-name $SVM_OPT_CFG

## First check - check our baseline performance

In [ ]:
!uv run cli python main.py compare-models \
    --experiment-id $EXPERIMENT_ID

In [8]:
data = """mean      mean      mean      mean
    0   svm-base  0.619658  0.627262  0.619658  0.621129
    1    svm-opt  0.692308  0.678169  0.692308  0.674845
    """
df = pd.read_csv(
        io.StringIO(data),
        skiprows=1,
        delim_whitespace=True,
        header=None,
        index_col=0,
        names=['model', 'accuracy', 'precision', 'recall', 'f1_score']
   )
df

/var/folders/fv/4xmxw7lx1l5d8bj50hs13zl00000gp/T/ipykernel_19386/3821847470.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


,model,accuracy,precision,recall,f1_score
0,svm-base,0.619658,0.627262,0.619658,0.621129
1,svm-opt,0.692308,0.678169,0.692308,0.674845


## Transformer model (Roberta)

### Micro-bert

[HuggingFace Link](https://huggingface.co/boltuix/bert-micro)

In [5]:
MICRO_BERT_BASE = "bert-micro-base"

In [ ]:
!uv run cli run-backtesting \
    --experiment-id $EXPERIMENT_ID \
    --model-config-name $MICRO_BERT_BASE

## Second check

In [ ]:
!uv run cli compare-models \
    --experiment-id $EXPERIMENT_ID

In [9]:
data = """mean      mean      mean      mean
0  bert-micro-base  0.536325  0.287644  0.536325  0.374458
1         svm-base  0.619658  0.627262  0.619658  0.621129
2          svm-opt  0.692308  0.678169  0.692308  0.674845
"""
df = pd.read_csv(
        io.StringIO(data),
        skiprows=1,
        delim_whitespace=True,
        header=None,
        index_col=0,
        names=['model', 'accuracy', 'precision', 'recall', 'f1_score']
   )
df

/var/folders/fv/4xmxw7lx1l5d8bj50hs13zl00000gp/T/ipykernel_19386/3415334600.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


,model,accuracy,precision,recall,f1_score
0,bert-micro-base,0.536325,0.287644,0.536325,0.374458
1,svm-base,0.619658,0.627262,0.619658,0.621129
2,svm-opt,0.692308,0.678169,0.692308,0.674845


Big dissapointment! Let's check more advanced model.

## Optimized mirco-bert 

In [ ]:
MICRO_BERT_LONG = "bert-micro-long"

In [ ]:
!uv run run-backtesting \
    --experiment-id $EXPERIMENT_ID \
    --model-config-name $MICRO_BERT_LONG

## Third check

In [ ]:
!uv run cli compare-models \
    --experiment-id $EXPERIMENT_ID

In [ ]:
data = """model_name  accuracy precision    recall  f1_score
                        mean      mean      mean      mean
0  bert-micro-base  0.536325  0.287644  0.536325  0.374458
1         svm-base  0.619658  0.627262  0.619658  0.621129
2          svm-opt  0.692308  0.678169  0.692308  0.674845
"""
df = pd.read_csv(
        io.StringIO(data),
        skiprows=1,
        delim_whitespace=True,
        header=None,
        index_col=0,
        names=['model', 'accuracy', 'precision', 'recall', 'f1_score']
   )
df

## Bigger model Roberta

In [10]:
ROBERTA_BASE = "roberta-base"

In [11]:
!uv run cli run-backtesting \
    --experiment-id $EXPERIMENT_ID\
    --model-config-name $ROBERTA_BASE

2025-10-18 16:55:30.603 | INFO     | mlops.runner:run_backtesting:58 - --- Backtesting Fold 0 ---
2025-10-18 16:55:30.603 | INFO     | mlops.runner:_load_model_from_config:28 - Loading model: RobertaModel from models.roberta.model
2025-10-18 16:55:34.793 | INFO     | mlops.runner:run_backtesting:68 - Training model...
2025-10-18 16:55:34.793 | INFO     | models.roberta.pipeline:run:15 - Preprocessing data for RoBERTa model...
Map: 100%|████████████████████████| 4205/4205 [00:00<00:00, 11619.62 examples/s]
2025-10-18 16:55:35.782 | INFO     | models.roberta.pipeline:run:41 - Preprocessing validation data...
Map: 100%|██████████████████████████| 468/468 [00:00<00:00, 11702.31 examples/s]
2025-10-18 16:55:35.843 | INFO     | models.roberta.pipeline:run:48 - Validation data preprocessing complete.
2025-10-18 16:55:35.843 | INFO     | models.roberta.pipeline:run:53 - Data preprocessing complete.
2025-10-18 16:55:35.843 | INFO     | models.roberta.pipeline:run:58 - Training RoBERTa model...


## Fourth check

In [12]:
!uv run cli compare-models \
    --experiment-id $EXPERIMENT_ID

2025-10-18 17:39:50.129 | WARNING  | evaluate:compare_models:56 - No results file found for run type 'performance_estimation' in experiment 'exp004'.
2025-10-18 17:39:50.131 | INFO     | evaluate:compare_models:79 - --- Model Comparison for Experiment 'exp004' ---
2025-10-18 17:39:50.132 | INFO     | evaluate:compare_models:82 - 
--- Run Type: backtesting ---
2025-10-18 17:39:50.132 | INFO     | evaluate:compare_models:91 - Summary for backtesting (train-val strategy):
2025-10-18 17:39:50.138 | INFO     | evaluate:compare_models:95 - 
        model_name  accuracy precision    recall  f1_score
                        mean      mean      mean      mean
0  bert-micro-base  0.536325  0.287644  0.536325  0.374458
1     roberta-base  0.816239  0.817263  0.816239  0.793307
2         svm-base  0.619658  0.627262  0.619658  0.621129
3          svm-opt  0.692308  0.678169  0.692308  0.674845


In [ ]:
data = """mean      mean      mean      mean
0  bert-micro-base  0.536325  0.287644  0.536325  0.374458
1     roberta-base  0.816239  0.817263  0.816239  0.793307
2         svm-base  0.619658  0.627262  0.619658  0.621129
3          svm-opt  0.692308  0.678169  0.692308  0.674845
"""
df = pd.read_csv(
        io.StringIO(data),
        skiprows=1,
        delim_whitespace=True,
        header=None,
        index_col=0,
        names=['model', 'accuracy', 'precision', 'recall', 'f1_score']
   )
df

/var/folders/fv/4xmxw7lx1l5d8bj50hs13zl00000gp/T/ipykernel_19386/3235625400.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


,model,accuracy,precision,recall,f1_score
0,bert-micro-base,0.536325,0.287644,0.536325,0.374458
1,roberta-base,0.816239,0.817263,0.816239,0.793307
2,svm-base,0.619658,0.627262,0.619658,0.621129
3,svm-opt,0.692308,0.678169,0.692308,0.674845
